## Maszynowa analiza danych 2022/2023
### Laboratorium 6
### Lime
Antoni Perużyński \\
Informatyka Stacjonarne \\

In [1]:
#%pip install interpret

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions

from sklearn.ensemble import RandomForestClassifier

from interpret.blackbox import LimeTabular
from interpret import show

from sklearn.preprocessing import StandardScaler
np.random.seed(222)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
dane = pd.read_csv("titanic.csv", index_col=0)
dane.head()

,gender,age,class,embarked,fare,sibsp,parch,survived
0,1,42.0,2,3,7.11,0,0,0
1,1,13.0,2,3,20.05,0,2,0
2,1,16.0,2,3,20.05,1,1,0
3,0,39.0,2,3,20.05,1,1,1
4,0,16.0,2,3,7.13,0,0,1


In [7]:
# dane["extra"] = 0
# dane["extra"][dane['gender']==1] = 1

In [8]:
# dane=dane.iloc[:, [1,0,2,3,4,5,6,8,7]]
# dane

# scaler = StandardScaler()
# dane.iloc[:,:-1] = scaler.fit_transform(dane.iloc[:,:-1])
# dane.head()

In [9]:
#dane = dane.drop('gender', axis=1)

In [10]:
dane.sort_values(by='age').head()
Sophia = dane.sort_values(by='age').iloc[:,:-1].head(1).values

In [11]:
dane[(dane['class']==4) & (dane['age']>35)].head()
Max = dane[(dane['class']==4) & (dane['age']>35)].iloc[:,:-1].head(1).values

In [12]:
dane[(dane['gender']==0) & (dane['age']>55)].head()
Maria = dane[(dane['gender']==0) & (dane['age']>55)].iloc[:,:-1].head(1).values

In [13]:
dane[(dane['gender']==1) & (dane['age']<25)].head()
David = dane[(dane['gender']==1) & (dane['age']<25)].iloc[:,:-1].head(1).values

In [14]:
X = dane.iloc[:,:-1]
y = dane['survived']

In [15]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X, y)

RandomForestClassifier()

In [16]:
Max[0]

array([ 1., 44.,  4.,  3.,  0.,  0.,  0.])

In [20]:
X_przypadki = np.array([Sophia[0], #Niemowlak 948
                        Max[0],    #Obsługa 653
                        [ 1., 44,  4.,  3.,  0.,  0.,  0.],
                        [ 0, 44,  4.,  3.,  0.,  0.,  0.],
                        Maria[0],   #Kobieta starsza 44
                        David[0]])  #Mężczyzna młody 1

y_przypadki = rf.predict(X_przypadki)

In [21]:
# X_przypadki = np.array([Sophia[0] #Niemowlak 948
# ])

# y_przypadki = rf.predict(X_przypadki)

In [22]:
lime_inter = LimeTabular(predict_fn=rf.predict_proba, data=X, random_state=100)

In [23]:
#?LimeTabular.explain_local

In [24]:
inter_lokal = lime_inter.explain_local(
    X_przypadki, y_przypadki, name='lime')
show(inter_lokal)

Sophia - niemowle. Widać, że zostało przydzielone do grupy 1, ze względu na zmienną survived tzn. udałoby mu się przeżyć. Prawodpodobieństwo, że ono przeżyje wynosi 87%. Z danych umieszczonych na wykresie można odczytać, że zmienna  tylko parch zwiększa wartość prawdopodobieństwa poza wyrazem wolnym. Zmniejszają natomiast wartość tego prawdopodobieństwa wszystkie inne zmienne. Największy wpływ mają zmienne fare (-0.067) oraz sibsp (-0.056). 

Max - osoba z obsługi. Widać, że został przydzielony do grupy 0, ze względu na zmienną survived tzn. nie udałoby mu się przeżyć. Prawodpodobieństwo, że przeżyje wynosi 13%, oczywiście, że nie przeżyje wynosi 87%. Z danych umieszczonych na wykresie można odczytać, że zmienna fare, sibsp, parch zwiększają wartość prawdopodobieństwa poza wyrazem wolnym. Zmniejszają natomiast wartość tego prawdopodobieństwa wszystkie inne zmienne. Największy wpływ miała zmienna fare współczynnik równy 0.06.

Maria - starsza kobieta. Widać, że została przydzielona do grupy 1, ze względu na zmienną survived tzn. udałoby się jej przeżyć. Prawodpodobieństwo, że przeżyje wynosi 93%, oczywiście. Z danych umieszczonych na wykresie można odczytać, że zmienna class, embarked, parch zwiększają wartość prawdopodobieństwa poza wyrazem wolnym. Zmniejszają natomiast wartość tego prawdopodobieństwa wszystkie inne zmienne. Najbardziej zwiększa zmienna class, ponieważ współczynnik przy tej zmiennej wynosi 0.11.

David - młody mężczyzna. Widać, że został przydzielony do grupy 0, ze względu na zmienną survived tzn. nie udałoby mu się przeżyć. Prawodpodobieństwo, że przeżyje wynosi 17%, oczywiście, że nie przeżyje wynosi 83%. Z danych umieszczonych na wykresie można odczytać, że zmienna fare, age zwiększają wartość prawdopodobieństwa poza wyrazem wolnym. Zmniejszają natomiast wartość tego prawdopodobieństwa wszystkie inne zmienne. Znacząco zmniejsza wartość zmienna fare, ponieważ współczynnik przy tej zmiennej wynosi -0.08. Co jest znacząco większe (co do modułu) od innych wartości ujemnych. A zwiększa zmienna sibsp, chociaż wartość dla tej osoby tej cechy wynosi tak naprawdę 0.